In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pylab as plt
import numpy as np

In [38]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"

In [165]:
def gen_data(N=100, d=10, low=0, high=10, target_idx=3):
    data = np.random.randint(low=low, high=high, size=(N,d))
    return data, data[:, 3]

In [166]:
train_data[0]

array([ 5, 10,  8, 17,  0,  5, 12,  3, 19, 14])

In [167]:
N = 5000
train_data, train_target = gen_data(N=N)
test_data, test_target = gen_data(N=N)

Input array: a

Fixed index: i

Experiment 1: Always predict entry in fixed index position: a $\rightarrow$ a[i] #current PC

Experiment 2: a $\rightarrow$ a[target_idx + a[i]] #going to correct instruction location - offset

Experiment 3: conditional on a[target_idx + a[i]] #increment to next PC

## Experiment 1

In [178]:
d = 10
low = 0
high = 10
n_hidden = 10
target_idx = 3

net = nn.Sequential(nn.Linear(d, n_hidden),
                    nn.ReLU(),
                    nn.Linear(n_hidden, high-low))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-2)

In [169]:
net(torch.from_numpy(train_data[0:5]).float()).shape

torch.Size([5, 10])

In [170]:
net = net.to(device)
N_epochs = 10000

train_data_torch = torch.from_numpy(train_data).float().to(device)
train_target_torch = torch.from_numpy(train_target).to(device)

def validate(net, test_data, test_target):
    
    test_data_torch = torch.from_numpy(test_data).float().to(device)
    test_target_torch = torch.from_numpy(test_target).float().to(device)

    test_pred = net(test_data_torch).argmax(dim=1)
    test_accuracy = (test_target_torch == test_pred).float().mean().to('cpu').detach().numpy()

    test_rmse = ((test_pred-test_target_torch)**2).mean().sqrt().to('cpu').detach().numpy()

    return test_accuracy, test_rmse

for i in range(N_epochs):
    pred = net(train_data_torch)
    loss = criterion(pred, train_target_torch)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        train_accuracy, train_rmse = validate(net, train_data, train_target)
        test_accuracy, test_rmse = validate(net, test_data, test_target)
        print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')

Epoch = 0 loss=3.0839340686798096 train_acc = 0.064 test_acc = 0.056 train_rmse = 5.120  test_rmse = 5.176
Epoch = 100 loss=1.172898292541504 train_acc = 0.603 test_acc = 0.604 train_rmse = 0.850  test_rmse = 0.856
Epoch = 200 loss=0.7593762278556824 train_acc = 0.745 test_acc = 0.746 train_rmse = 0.545  test_rmse = 0.542
Epoch = 300 loss=0.47754454612731934 train_acc = 0.943 test_acc = 0.941 train_rmse = 0.240  test_rmse = 0.242
Epoch = 400 loss=0.31051865220069885 train_acc = 0.992 test_acc = 0.991 train_rmse = 0.092  test_rmse = 0.093
Epoch = 500 loss=0.20938293635845184 train_acc = 0.999 test_acc = 0.997 train_rmse = 0.042  test_rmse = 0.053
Epoch = 600 loss=0.1411212533712387 train_acc = 1.000 test_acc = 0.999 train_rmse = 0.000  test_rmse = 0.035
Epoch = 700 loss=0.09734237939119339 train_acc = 1.000 test_acc = 0.999 train_rmse = 0.000  test_rmse = 0.032
Epoch = 800 loss=0.06903334707021713 train_acc = 1.000 test_acc = 0.999 train_rmse = 0.000  test_rmse = 0.035
Epoch = 900 loss=

Epoch = 7400 loss=8.84419641806744e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 7500 loss=8.329381671501324e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 7600 loss=7.863183418521658e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 7700 loss=7.400791946565732e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 7800 loss=6.980789476074278e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 7900 loss=6.578118336619809e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 8000 loss=6.233590102056041e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 8100 loss=5.867405707249418e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  test_rmse = 0.028
Epoch = 8200 loss=5.5152289860416204e-05 train_acc = 1.000 test_acc = 1.000 train_rmse = 0.000  t

In [179]:
#weights are not "peaky"

for _ in range(n_hidden):
    l = list(net.parameters())[0][_]
    
    print('---')
    print(l)
    print(l[target_idx].abs() / l.abs().min())
    
print('***')
for _ in range(high-low):
    l = list(net.parameters())[2][_]
    
    print('---')
    print(l)
    
    print(l[target_idx].abs() / l.abs().min())

---
tensor([-0.0800, -0.1408, -0.1178, -0.2991,  0.1991, -0.1334, -0.2901, -0.2046,
         0.0387, -0.1757], grad_fn=<SelectBackward0>)
tensor(7.7322, grad_fn=<DivBackward0>)
---
tensor([-0.2968,  0.2352, -0.1972, -0.1490,  0.0768, -0.0311,  0.0108,  0.1800,
        -0.0566,  0.2452], grad_fn=<SelectBackward0>)
tensor(13.7839, grad_fn=<DivBackward0>)
---
tensor([-0.2986, -0.1014,  0.3023,  0.0371,  0.3115,  0.2062, -0.1687, -0.1000,
        -0.3145, -0.2613], grad_fn=<SelectBackward0>)
tensor(1., grad_fn=<DivBackward0>)
---
tensor([ 0.2468,  0.2756, -0.2339, -0.2362,  0.2236, -0.0384, -0.0148,  0.2684,
        -0.2050,  0.0962], grad_fn=<SelectBackward0>)
tensor(15.9630, grad_fn=<DivBackward0>)
---
tensor([ 0.0620, -0.1065, -0.1631,  0.3001, -0.1489, -0.0319, -0.2194,  0.1046,
        -0.0094,  0.2317], grad_fn=<SelectBackward0>)
tensor(31.8040, grad_fn=<DivBackward0>)
---
tensor([ 0.0090,  0.2770, -0.2179, -0.2923, -0.0290, -0.3144,  0.0888, -0.1450,
         0.2830, -0.2811], grad_

In [183]:
list(net.parameters())[0].sum(dim=0)

tensor([-0.9449,  0.2442, -0.8069, -0.2262,  1.0153, -0.5774, -0.9757, -0.0368,
        -0.2862,  0.2354], grad_fn=<SumBackward1>)

In [184]:
list(net.parameters())[0].sum(dim=1)

tensor([-1.2038,  0.0172, -0.3875,  0.3824,  0.0191, -0.6219, -0.6494,  0.2659,
        -0.0248, -0.1566], grad_fn=<SumBackward1>)

In [142]:
test_data_torch[2]

tensor([ 8., 17.,  9., 17.,  4., 15., 11., 17.,  7., 12.], device='cuda:0')

In [153]:
np.matmul(np.array(list(net.parameters())[0].cpu().detach()), test_data[2]) + np.array(list(net.parameters())[1].cpu().detach())

array([  3.4826468 ,   2.28380267, -36.50725102,   6.21199597,
         3.45775655, -16.23347923, 390.02531135,  16.91296743,
       141.38191938,   2.78582338,  14.43411553,   2.15547779,
       -29.41698521, -22.8365572 ,   3.5040608 , -21.2656742 ,
       -35.28759741, -42.19878227, -17.10998713, -24.47549912])

### Experiments 1X:

1a. what happens when number of hidden nodes is decreased from 20?

1b. 2d (t-sne) visualizations

    raw data -> t-sne (colors are the target value)
    
    raw data -> nn -> take the activations (20 intermediate values) -> t-sne (colors are the target value)
    
1c. interpreting nns: which input element has the most effect on the output

1d. paths from each input node to each output node

1e. compute derivatives of output nodes w.r.t. input nodes and look at max absolute value

1f. replace net by convolutional net and check weights